3/4　進捗報告と本日の質問

＜進捗＞
ロジスティック回帰、SVM,木、ランダムフォレスト、ｱﾀﾞﾌﾞｰｽﾄのモデルそれぞれで、正則化・標準化・ハイパーパラメータをGridsearchで探索の場合での精度を確かめてみました。
組み込み法で躓いています。

＜本日お伺いしたいこと＞  
1.正則化について  
木・ランダムフォレスト・ｱﾀﾞﾌﾞｰｽﾄは不純度や情報量のばらつきでモデルを構築するので、正則化の考え方はない？  


2.標準化について  
数値の「goal」「days」にしか行わず、get_dummiesで処理したカテゴリ変数には行いませんでした。その考え方であっているか？  

3.Slackでお伺いした、SVMの処理時間がかかる件  
ハイパーパラメータ探索を全体のデータの5%、モデル構築のデータを全体の20%で行いました。このような考え方であっているか？「【林優夏】3月4日ビデオチャット用　SVM（標準化＋params）」のnotebookです。  

4.組み込み法について  
エラーが出てしまう（このnotebook）  

5.標準化・組み込み法・ハイパーパラメータ探索の順序について  
標準化を行い、組み込み法で変数を選択→その後ハイパーパラメータ探索で正則化をl1にするかl2にするか選んだり、他のパラメータを設定する、という基本的な流れでいいか  

6.name列について
「【林優夏】3月4日ビデオチャット用　nameの考察」のnotebookの様な考え方をしてみましたのでご意見伺いたいです

①データの読み込み・前処理

In [32]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns
matplotlib.style.use('ggplot')

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
%matplotlib inline

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

In [33]:
df = pd.read_csv(r"C:\Users\yuka0\Documents\skillupai\ML\DAY1\homework\ks-projects-201801.csv")

In [34]:
#募集期間の加工
df['deadline'] = pd.to_datetime(df["deadline"])
df["launched"] = pd.to_datetime(df["launched"])

df["days"] = (df["deadline"] - df["launched"]).dt.days

In [37]:
#stateが成功と失敗以外のデータを削除
#ID削除
#nameとcategoryも削除
#daysを募集期間で出したのでdf["deadline"]とdf["launched"]も削除
#backers,pledgedと、usd pledgedより右はモデルにいれちゃだめだと思うので削除

In [38]:
#データの削除・加工
df = df[(df["state"] == "successful") | (df["state"] == "failed")]
df = df.drop(["ID","name","deadline","launched","backers","pledged","usd pledged","usd_pledged_real","usd_goal_real"], axis=1)
df["state"] = df["state"].replace("failed",0)
df["state"] = df["state"].replace("successful",1)

④カテゴリ変数処理

In [42]:
#カテゴリ変数処理
df = pd.get_dummies(df,drop_first = True)

In [43]:
#標準化
stdsc = StandardScaler()
df["goal"] = stdsc.fit_transform(df[["goal"]].values)
df["days"] = stdsc.fit_transform(df[["days"]].values)

C:\Users\yuka0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\yuka0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


②データ分割・変数選択（組み込み法）

In [44]:
#データ分割
train_data = df.drop("state", axis=1)
y = df["state"].values
X = train_data.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

組み込み法

In [45]:
estimator = LassoCV(normalize = True, cv = 10)
sfm = SelectFromModel(estimator, threshold = 1e-5)
sfm.fit(X_train_selected,y_train)

SelectFromModel(estimator=LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False),
        max_features=None, norm_order=1, prefit=False, threshold=1e-05)

In [46]:
X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

ValueError: X has a different shape than during fitting.

In [ ]:
classifier = SGDClassifier()
classifier.fit(X_train_selected, y_train)
classifier.score(X_test_selected, y_test)

In [ ]:
# get_support関数で使用する特徴のインデックスを使用
# Trueになっている特徴が使用する特徴
sfm.get_support()

In [ ]:
# 削除すべき特徴の名前を取得 
removed_idx = ~sfm.get_support()
train_data.columns[removed_idx]

③ハイパーパラメータ設定

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

parameters = {'penalty':['l1', 'l2'], 'max_iter':[5000,30000], 'tol':[1e-10,1e-1]} # ここを編集する
model = classifier
clf = GridSearchCV(model, parameters, cv=3,)
clf.fit(X_train, y_train)
print(clf.best_params_, clf.best_score_)